<a href="https://colab.research.google.com/github/wcjona/BMEN_415_Final_Project/blob/main/Regression_Models/FN_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
df = pd.read_csv("/content/drive/MyDrive/BMEN_415_Group10/ML_Models/Dataset/Volumetric_features.csv")
df = df.drop(columns=["S.No", "dataset", "Left-WM-hypointensities", "Right-WM-hypointensities", "Left-non-WM-hypointensities", "Right-non-WM-hypointensities"])
df = df.drop_duplicates()
df = df.dropna()
y = df["Age"]
x = df.loc[:,df.columns!="Age"]
x_no = (x-x.min())/(x.max()-x.min())
x_st = (x-x.mean())/x.std()
x_train_no, x_test_no, y_train_no, y_test_no = train_test_split (x_no, y, test_size = 0.2, random_state = 142)
x_train_st, x_test_st, y_train_st, y_test_st = train_test_split (x_st, y, test_size = 0.2, random_state = 142)

In [ ]:
def process(model):
  m_no = model.fit(x_train_no,y_train_no)
  a = m_no.score(x_test_no,y_test_no)
  m_st = model.fit(x_train_st,y_train_st)
  b = m_st.score(x_test_st,y_test_st)
  if a > b:
    print(round(a,4), 'using normalization')
    mod = model.fit(x_train_no,y_train_no)
    y_pred = mod.predict(x_test_no)
    print(round(mean_squared_error(y_test_no,y_pred),4))
  else:
    print(round(b,4), 'using standardization')
    mod = model.fit(x_train_st,y_train_st)
    y_pred = mod.predict(x_test_st)
    print(round(mean_squared_error(y_test_st,y_pred),4))

In [ ]:
#SGD
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(random_state=42)
process(sgd)

0.8459 using standardization
61.1764


In [ ]:
#Boosted
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=200,max_depth=5,random_state=42)
process(gbr)

0.8868 using standardization
44.9408


In [ ]:
gbr_vr = GradientBoostingRegressor(n_estimators=110,max_depth=5,random_state=42)
process(gbr_vr)

0.884 using standardization
46.0585


In [ ]:
#Voting Regressor will use 2 regressors w highest R2
#Boosted, Random Forest

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=42)
process(rfr)

0.8695 using standardization
51.8245


In [ ]:
#VR with Boosted and Random Forest
from sklearn.ensemble import VotingRegressor
vr = VotingRegressor([("gbr_vr",gbr_vr),("rfr",rfr)])
process(vr)

0.8821 using standardization
46.8046
